In [1]:
import torch
import matchzoo as mz
import nltk

#nltk.set_proxy('SYSTEM PROXY')
nltk.download('punkt')

ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.MeanAveragePrecision()
]

[nltk_data] Downloading package punkt to /Users/shanewang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
valid_pack = mz.datasets.wiki_qa.load_data('dev', task=ranking_task)

In [3]:
preprocessor = mz.models.ArcI.get_default_preprocessor()
train_processed = preprocessor.fit_transform(train_pack)
valid_processed = preprocessor.transform(valid_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7177.35it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3777.93it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1248675.47it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5643.89it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4071.07it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 484993.75it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 296/296 [00:00<00:00, 6288.25it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2708/2708 [00:00<00

In [4]:
trainset = mz.dataloader.Dataset(
    data_pack=train_processed,
    mode='pair'
)
validset = mz.dataloader.Dataset(
    data_pack=valid_processed,
    mode='point'
)

In [5]:
padding_callback = mz.models.ArcI.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=32,
    stage='train',
    callback=padding_callback
)
validloader = mz.dataloader.DataLoader(
    dataset=validset,
    batch_size=32,
    stage='dev',
    callback=padding_callback
)

In [6]:
model = mz.models.ArcI()
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = 100
model.params['embedding_input_dim'] = preprocessor.context['embedding_input_dim']
model.guess_and_fill_missing_params()
model.build()

Parameter "embedding_input_dim" set to 300.
Parameter "embedding_output_dim" set to 300.


In [7]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=validloader,
    epochs=10
)

trainer.run()

RuntimeError: index out of range: Tried to access index 20817 out of table with 299 rows. at /Users/distiller/project/conda/conda-bld/pytorch_1573049308345/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [8]:
mz.__version__

'1.1'